In [4]:
import requests
import lxml.html as lh
import pandas as pd
import re 
import csv

In [100]:
def getApps(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    text = doc.text_content()
    hasNationalTeam = "Appearances and goals by national team and year" in text
    print("Appearances and goals by national team and year" in text)
    if hasNationalTeam:
        1
    else:
        return 0

# getApps('https://en.wikipedia.org/wiki/James_Tarkowski')

True


In [5]:
def getNationalTeamTable(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tables = doc.xpath('//table')
    for table in tables:
        captions = table.xpath('//caption')
        for caption in captions:
            text = caption.text_content()
            if "Appearances and goals by national team and year" in text:
                return table

# TODO: Better Look Up
def getAppsForPlayer(wiki_name):
    url = 'https://en.wikipedia.org/wiki/'+wiki_name
    table = getNationalTeamTable(url)
    if table:
        rows = table.xpath('//tr')
        i=0
        for row in rows:
            i=i+1
            if "Total" in row.text_content():
                if len(row) == 3:
                    total = row[1]
                    return int(total.text_content()), url
#     print('Error Getting Apps for: ' + url)
    return -1 ,url

In [10]:
def getPlayerRowsV2(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 14:
            name_cell = row[3]
            nationality_cell = row[2]
            text = name_cell.text_content().strip()
            skip = text.isnumeric() or len(text) == 0 or text == 'Name'
            if not skip:
                playerRows.append((name_cell,nationality_cell))
    if len(playerRows) == 0:
        print('Error Getting Players from: ' + url)
    else:
        print('Jheeeze found {} players'.format(len(playerRows)))
    return playerRows

def getPlayerRowsV1(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 12:
            name_cell = row[3]
            nationality_cell = row[2]
            text = name_cell.text_content().strip()
            skip = text.isnumeric() or len(text) == 0 or text == 'Name'
            if not skip:
                playerRows.append((name_cell,nationality_cell))
    if len(playerRows) == 0:
        print('Error Getting Players from: ' + url)
    else:
        print('Jheeeze found {} players'.format(len(playerRows)))
    return playerRows

def addPlayerApps(display_name,wiki_na):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [81]:
def notAPhoto(x):
    return "upload" not in x[2]

def getWikiLink(cell):
    allLinks = list(cell.iterlinks())
    goodLinks = list(filter(lambda x: notAPhoto(x), allLinks))
    if len(goodLinks)==0:
        return "ERROR: No Link"
    else:
        return goodLinks[0][2].replace('/wiki/','')

def getWikiString(string):
    return string.replace('https://en.wikipedia.org/wiki/','')

def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
    return display_name, getWikiLink(player), getWikiLink(country_cell)

In [86]:
def appearancesForPlayer(wiki_name):
    url = 'https://en.wikipedia.org/wiki/'+wiki_name
    table = getNationalTeamTable(url)
    if table:
        rows = table.xpath('//tr')
        i=0
        for row in rows:
            i=i+1
            if "Total" in row.text_content():
                if len(row) == 3:
                    total = row[1]
                    return int(total.text_content()), url
    return -1 ,url

def validPlayerRow(row):
    skip = False
    goodLength = len(row) in [12,13,14]
    if goodLength:
        name_cell = row[3]
        nationality_cell = row[2]
        text = name_cell.text_content().strip()
        skip = text.isnumeric() or len(text) == 0 or text == 'Name'
    return goodLength and not skip

def getRows(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    allRows = list(doc.xpath('//tr'))
#     print('All Rows Length is: ',len(allRows))
    goodRows = list(filter(lambda x: validPlayerRow(x), allRows))
#     print('Good Rows Length is: ',len(goodRows))
    for row in goodRows:
        name = row[3].text_content().strip()
        wikiName = getWikiLink(row[3])
        country = getWikiLink(row[2])
        (apps,url) = appearancesForPlayer(wikiName)
        print(name,wikiName,country,apps)
    

getRows('https://en.wikipedia.org/wiki/2019%E2%80%9320_Burnley_F.C._season')

<ipython-input-86-c6a0fadeed20>:4: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if table:


Nick Pope Nick_Pope_(footballer) England 4
Matthew Lowton Matthew_Lowton England -1
Charlie Taylor Charlie_Taylor_(footballer,_born_1993) England -1
Jack Cork Jack_Cork England 1
James Tarkowski James_Tarkowski England 2
Ben Mee Ben_Mee England -1
Jóhann Berg Guðmundsson J%C3%B3hann_Berg_Gu%C3%B0mundsson Iceland 77
Danny Drinkwater * Danny_Drinkwater England 3
Josh Brownhill Josh_Brownhill England -1
Chris Wood Chris_Wood_(footballer,_born_1991) New_Zealand 57
Ashley Barnes Ashley_Barnes England -1
Dwight McNeil Dwight_McNeil England -1
Robbie Brady Robbie_Brady Republic_of_Ireland 54
Jeff Hendrick Jeff_Hendrick Republic_of_Ireland 60
Ben Gibson Ben_Gibson England -1
Bailey Peacock-Farrell Bailey_Peacock-Farrell Northern_Ireland 18
Steven Defour Steven_Defour Belgium 52
Ashley Westwood Ashley_Westwood_(footballer,_born_1990) England -1
Jay Rodriguez Jay_Rodriguez England 1
Joe Hart Joe_Hart England 75
Erik Pieters Erik_Pieters Netherlands 18
Aaron Lennon Aaron_Lennon England 21
Phil Ba

In [43]:
rows = getPlayerDetails('https://en.wikipedia.org/wiki/2019%E2%80%9320_Burnley_F.C._season',14)

Nick Pope Nick_Pope_(footballer)
Matthew Lowton Matthew_Lowton
Charlie Taylor Charlie_Taylor_(footballer,_born_1993)
Jack Cork Jack_Cork
James Tarkowski James_Tarkowski
Ben Mee Ben_Mee
Jóhann Berg Guðmundsson J%C3%B3hann_Berg_Gu%C3%B0mundsson
Danny Drinkwater * Danny_Drinkwater
Josh Brownhill Josh_Brownhill
Chris Wood Chris_Wood_(footballer,_born_1991)
Ashley Barnes Ashley_Barnes
Dwight McNeil Dwight_McNeil
Robbie Brady Robbie_Brady
Jeff Hendrick Jeff_Hendrick
Ben Gibson Ben_Gibson
Bailey Peacock-Farrell Bailey_Peacock-Farrell
Steven Defour Steven_Defour
Ashley Westwood Ashley_Westwood_(footballer,_born_1990)
Jay Rodriguez Jay_Rodriguez
Joe Hart Joe_Hart
Erik Pieters Erik_Pieters
Aaron Lennon Aaron_Lennon
Phil Bardsley Phil_Bardsley
Matěj Vydra Mat%C4%9Bj_Vydra
Kevin Long Kevin_Long_(footballer)
Adam Legzdins Adam_Legzdins
Max Thompson Max_Thompson_(footballer,_born_2002)
Jimmy Dunne Jimmy_Dunne_(footballer,_born_1997)
Bobby Thomas ERROR
Lewis Richardson ERROR
Lukas Jensen Lukas_Jensen

In [4]:
def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [5]:
def getAppearanceArray(url):
    playerRows = getPlayerRows(url)
    linksArray = [getWikiDetails(player) for player in playerRows]
    appsArray = [addPlayerApps(names) for names in linksArray]
    return appsArray

In [6]:
def getTeams(url):
    teams = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 4:
            cell = row[0]
            text = cell.text_content().strip()
            skip = text == 'Team' or text == 'Rank' or text.isnumeric()
            if not skip:
                teams.append(cell)
    return teams


def addTeamToEachItem(player,teamName,squad_url):
    (display_name, wiki_name, country, apps, url) = player
    return teamName,squad_url,display_name, wiki_name, country, apps, url

In [7]:
def fuckedIt(name):
    return 'Fucked It','should be wiki link for'+name,'but Im weird', None, None, None, None

In [8]:
def toDf(data):
    return pd.DataFrame(data, columns = ['teamName','squad_url','display_name', 'wiki_name', 'country', 'apps', 'url']) 

In [9]:
def getDataFrameForLeague(url):
    teams = getTeams(url)
    dfData = []
    for team in teams:
        teamDisplayName = team.text_content()
        wiki_name = getWikiLink(team)
        if wiki_name == 'ERROR':
            dfData = dfData + [fuckedIt(teamDisplayName)]
        else:
            squad_url = 'https://en.wikipedia.org/wiki/2019-20_'+wiki_name+'_season'
            print('Doing Look up for: '+ teamDisplayName)
            appearanceArray = getAppearanceArray(squad_url)
            updatedArray = [addTeamToEachItem(player,teamDisplayName,squad_url) for player in appearanceArray]
            dfData = dfData + updatedArray
    return toDf(dfData)

In [10]:
def writeDataToFile(df,fileName):
    df.to_csv('../data/'+fileName,index=False)
    print(df.head())

In [11]:
def getLeagueList(pathToFile):
    with open(pathToFile, newline='') as csvfile:
        leagues = list(csv.reader(csvfile))
    return leagues

In [12]:
def flatten(object):
    country = object["country"]
    league = getWikiString(object["url"])
    data = object["data"]
    data['country']=country
    data['league']=league
    return data

def getLeagueDF(league):
    country = league[0]
    url = league[1]
    leagueData = getDataFrameForLeague(url)
    thisDict = {
        "country":country,
        "url":url,
        "data":leagueData
    }
    return flatten(thisDict)

def collapseDfs(arrayDf):
    base = arrayDf[0]
    arrayDf.pop(0)
    length = len(arrayDf)
    for i in range(length):
        base = base.append(arrayDf[i])
    return base

In [15]:
def fullThing(leaguesFile,outputName):
    leagues = getLeagueList(leaguesFile)
    dataFrames = [getLeagueDF(league) for league in leagues]
    finalData = collapseDfs(dataFrames)
    writeDataToFile(finalData,outputName)


In [17]:
fullThing('../data/England.csv','mark1.csv')

Doing Look up for: Alavés
Jheeeze found 1 players
Doing Look up for: Athletic Bilbao
Jheeeze found 1 players
Doing Look up for: Atlético Madrid
Jheeeze found 44 players


<ipython-input-2-5abd54b62d75>:16: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if table:


Doing Look up for: Barcelona
Jheeeze found 40 players
Doing Look up for: Celta Vigo
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_RC_Celta_de_Vigo_season
Doing Look up for: Eibar
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_SD_Eibar_season
Doing Look up for: Espanyol
Jheeeze found 37 players
Doing Look up for: Getafe
Jheeeze found 31 players
Doing Look up for: Granada
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Granada_CF_season
Doing Look up for: Leganés
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_CD_Legan%C3%A9s_season
Doing Look up for: Levante
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Levante_UD_season
Doing Look up for: Mallorca
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_RCD_Mallorca_season
Doing Look up for: Osasuna
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_CA_Osasuna_season
Doing Look up for: Real Betis
Error Getting Players 

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Kayserispor_season
Doing Look up for: Konyaspor

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Konyaspor_season
Doing Look up for: Sivasspor

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Sivasspor_season
Doing Look up for: Trabzonspor

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Trabzonspor_season
Doing Look up for: Yeni Malatyaspor

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Yeni_Malatyaspor_season
Doing Look up for: Admira Wacker Mödling

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FC_Admira_Wacker_M%C3%B6dling_season
Doing Look up for: Austria Wien

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FK_Austria_Wien_season
Doing Look up for: LASK

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_LASK_season
Doing Look up for: Rapid Wien

Jheeeze found 32 players
Doing Look up for: 

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FK_Radnik_Surdulica_season
Doing Look up for: Red Star

Jheeeze found 36 players
Doing Look up for: Spartak

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FK_Spartak_Subotica_season
Doing Look up for: Vojvodina

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FK_Vojvodina_season
Doing Look up for: Voždovac

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_FK_Vo%C5%BEdovac_season
Doing Look up for: TSC

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_TSC_Ba%C4%8Dka_Topola_season
Doing Look up for: Dinamo Zagreb

Jheeeze found 1 players
Doing Look up for: Stadion Maksimir

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Stadion_Maksimir_season
Doing Look up for: 

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_File:Stadion_Maksimir_in_February_2019.jpg_season
Doing Look up for: Osijek

Error Getting Players from: h

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Huachipato_season
Doing Look up for: O'Higgins

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_O%27Higgins_F.C._season
Doing Look up for: Palestino

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Club_Deportivo_Palestino_season
Doing Look up for: Unión Española

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Uni%C3%B3n_Espa%C3%B1ola_season
Doing Look up for: Unión La Calera

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Uni%C3%B3n_La_Calera_season
Doing Look up for: Universidad Católica

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Club_Deportivo_Universidad_Cat%C3%B3lica_season
Doing Look up for: Universidad de Chile

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Club_Universidad_de_Chile_season
Doing Look up for: Universidad de Concepción

Error Getting Players from: https://en.wikipedia.org/wiki/2019-2

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Western_Sydney_Wanderers_FC_season
Doing Look up for: Western United

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Western_United_FC_season
Doing Look up for: Western United

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Western_United_FC_season
Doing Look up for:  Dino Djulbic
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Australia_season
Doing Look up for:  Harrison Delbridge
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Australia_season
Doing Look up for:  Steven Taylor
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_England_season
Doing Look up for:  Luke DeVere
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Australia_season
Doing Look up for:  Matthew Millar
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Australia_season
Doing Look up for:  Kye Rowles
Error Getting Players from: ht